In [ ]:
# Sources:
# https://colab.research.google.com/github/araffin/rl-tutorial-jnrr19/blob/master/5_custom_gym_env.ipynb
# https://deeplizard.com/learn/video/HGeI30uATws

# Import Dependencies

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import osmnx as ox

import os
import random
import time
import math
import gym
import csv
import folium

from gym import Env
from gym import utils
from gym.spaces import Discrete, Box
from stable_baselines3.common.env_checker import check_env
from IPython.display import clear_output
from shapely.geometry import Polygon

# Load Road Network as Graph

In [ ]:
boundary = Polygon([(-122.75593463884833, 38.48043857931626), (-122.75593463884833, 38.44276212966626), 
                    (-122.71602508838065, 38.44276212966626), (-122.71602508838065, 38.48043857931626)])

In [ ]:
graph = ox.graph.graph_from_polygon(boundary, network_type = 'drive')
graph = graph.to_undirected()

graph = ox.speed.add_edge_speeds(graph, precision = 3)
graph = ox.speed.add_edge_travel_times(graph, precision = 3)

nodes, edges = ox.graph_to_gdfs(graph)

In [ ]:
nodes, edges = ox.graph_to_gdfs(graph)

In [ ]:
edges = edges.drop(columns = ['osmid', 'bridge', 'oneway', 'lanes', 'maxspeed', 
                              'geometry', 'ref', 'name'])

In [ ]:
edges

In [ ]:
m = ox.plot_graph_folium(graph, popup_attribute = 'travel_time', weight = 3, color = '#3498DB')
m

In [ ]:
id_list = []

for n in range(len(graph)):
    id_list.append(nodes.iloc[n].name)

# Define Sample Paths

In [ ]:
colors = ['#E74C3C', '#3498DB', '#F7DC6F']

sample_paths = []

# Path 0: shortest path by length
sample_paths.append(list(ox.distance.shortest_path(graph, 56102877, 1965862095, weight = 'length')))

# Path 1: shortest path by travel time
sample_paths.append(list(ox.distance.shortest_path(graph, 56102877, 1965862095, weight = 'travel_time')))

# Path 2: manually defined path
sample_paths.append([56102877, 6940021052, 56102875, 56043223, 56102872, 55973175, 56004921, 55950049, 55950051,
                     55950052, 55950053, 56125038, 56078315, 56040934, 56084803, 56084797, 7049377524, 7049377523,
                     7049377522, 56084795, 7049377535, 7049377534, 7049377521, 7049377518, 56116115, 56116919,
                     56116921, 56116923, 8268741787, 56019308, 56021878, 7204303428, 56116927, 56157636, 56034980,
                     56129707, 56054706, 56097568, 56129711, 56117906, 56140110, 56055340, 56173132, 56102741,
                     56109019, 56027403, 56043321, 56101250, 56040774, 56017692, 56040783, 1965862095])

ox.plot.plot_graph_routes(graph, sample_paths, route_colors = colors)

# Create Environment

In [ ]:
class PathEnv(gym.Env):
    
    # Define metadata
    metadata = {'render.modes': ['human']}
    
    # Set path to test: 0, 1, 2
    PATH = sample_paths[2]
    
    def __init__(self):
        super(PathEnv, self).__init__()
        
        # Initialize agent starting node
        self.agent_node = 489
        
        # Initialize previous agent node
        self.prev_agent_node = self.agent_node
        
        # Initialize agent path
        self.agent_path = [self.agent_node]
        
        # Initialize goal node
        self.goal_node = 752
        
        # Initialize goal time
        self.goal_time = self.get_path_time(self.PATH)
        
        # Initialize time elapsed
        self.time_elapsed = 0
        
        # Define action and observation spaces
        n_actions = len(graph)
        self.action_space = Discrete(n_actions)
        self.observation_space = Discrete(n_actions)
    
    def step(self, action):
        reward = 0
        done = False

        if action in self.get_neighbor_indices():
            # Save previous distance to goal
            prev_dist = self.get_distance_to_goal()
            
            # Update previous agent node, agent node
            self.prev_agent_node = self.agent_node
            self.agent_node = action

            # Update agent path
            self.agent_path.append(action)

            # Update time elapsed
            self.time_elapsed += (graph.edges[(id_list[self.prev_agent_node], id_list[self.agent_node], 0)]['travel_time'] + 30) / 60
            
            # Get new distance to goal
            dist = self.get_distance_to_goal()
            
            if dist < prev_dist:
                # Reward for getting closer to goal
                reward = 1 
            else:
                # Penalize for going farther away from goal
                reward = -1
            
        else:
            # End episode if agent goes in a loop
            if action == -1:
                done = True
                neighbors = self.get_neighbor_indices()
                
                if len(neighbors) == 1:
                    if(neighbors[0] == self.prev_agent_node):
                        # Penalize for reaching a dead end
                        reward = -100
            else:
                print('Illegal action')
                done = True


        if self.goal_reached():
            done = True
            
            # Reward for getting close to goal time
            error = abs(self.goal_time - self.time_elapsed)
            
            if error < 2.5:
                reward += 10000
            
        # Set placeholder for info
        info = {}
        
        # Return step information
        n_state = self.agent_node
        return n_state, reward, done, info

    def render(self, mode = 'human'):
        if mode != 'human':
            raise NotImplementedError()
        
        ox.plot.plot_graph_route(graph, self.get_id_path(self.agent_path))
        
    def reset(self):
        
        # Initialize agent starting node
        self.agent_node = 489
        
        # Initialize previous agent node
        self.prev_agent_node = self.agent_node
        
        # Initialize agent path
        self.agent_path = [self.agent_node]
        
        # Initialize goal node
        self.goal_node = 752
        
        # Initialize goal time
        self.goal_time = self.get_path_time(self.PATH)
        
        # Initialize time elapsed
        self.time_elapsed = 0
        
        n_state = self.agent_node
        
        return n_state
    
    # Get indices of neighbors to current node
    def get_neighbor_indices(self):
        id = id_list[self.agent_node]

        neighbors = list(graph.neighbors(id))
        neighbor_indices = []

        for i in range(len(id_list)):
            for n in neighbors:
                if n == id_list[i]:
                    neighbor_indices.append(i)

        return neighbor_indices
    
    # Get ID path from index path
    def get_id_path(self, path):
        id_path = path.copy()
        
        for n in range(len(id_path)):
            id_path[n] = id_list[id_path[n]]
        
        return id_path
    
    # Get index path from ID path
    def get_index_path(self, path):
        index_path = path.copy()
        
        for n in range(len(index_path)):
            index_path[n] = id_list.index(index_path[n])
        
        return index_path
    
    # Get path time of given path
    def get_path_time(self, path):
        path_time = 0
        
        for n in range(len(path) - 1):
            path_time += (graph.edges[(path[n], path[n + 1], 0)]['travel_time'] + 30) / 60
        
        return path_time
    
    # Get path distance of given path
    def get_path_distance(self, path):
        path_dist = 0
        
        for n in range(len(path) - 1):
            path_dist += graph.edges[(path[n], path[n + 1], 0)]['length']
        
        return path_dist
    
    # Get path accuracy info
    def get_path_accuracy(self, s, a):
        matching_nodes = 0
        
        for node in s:
            if node in a:
                matching_nodes += 1
        
        print('Matching nodes: {} / {}'.format(matching_nodes, len(s)))
        print('Length difference:', abs(len(s) - len(a)))
        
        accuracy_score = (matching_nodes / len(s)) * 100 - abs(len(s) - len(a))
        
        print('Accuracy score:', accuracy_score)
    
    def get_matching_nodes(self, s, a):
        
        matching_nodes = 0
        
        for node in s:
            if node in a:
                matching_nodes += 1
        
        return matching_nodes / len(s)
    
    def get_length_diff(self, s, a):
        return abs(len(s) - len(a))
    
    # Get agent's distance to goal
    def get_distance_to_goal(self):
        p = ox.distance.shortest_path(graph, id_list[self.agent_node], id_list[self.goal_node], weight = 'length')
        
        return self.get_path_distance(p)

    def get_possible_actions(self):
        neighbors = self.get_neighbor_indices()
        tmp = neighbors.copy()

        for n in neighbors:
            if n in self.agent_path:
                tmp.remove(n)
        
        return tmp
    
    def get_q_vals(self):
        q_vals = []
        possible_actions = self.get_possible_actions()
        
        for a in possible_actions:
            q_vals.append(q_table[state, a])
        
        return q_vals
    
    def get_best_action(self):
        possible_actions = self.get_possible_actions()
        
        if not possible_actions:
            return -1
        else:
            q_vals = self.get_q_vals()
        
        return possible_actions[random.choice(np.argwhere(q_vals == np.max(q_vals)).flatten().tolist())]

    # Returns true if agent has reached goal
    def goal_reached(self):
        if self.agent_node == self.goal_node:
            return True
        else:
            return False

# Validate Environment

In [ ]:
env = PathEnv()
check_env(env, warn = True)

In [ ]:
env.observation_space.sample()

In [ ]:
env.reset()
env.render()

# Test Random Environment

In [ ]:
env = PathEnv()

episodes = 10
max_steps = 100
history = []

for episode in range(1, episodes + 1):
    obs = env.reset()
    done = False
    score = 0
    
    clear_output(wait = True)
    env.render()
        
    time.sleep(0.5)
    
    for step in range(max_steps):
        possible_actions = env.get_possible_actions()
        
        if not possible_actions:
            obs, reward, done, info = env.step(-1)
        else:
            obs, reward, done, info = env.step(random.choice(possible_actions))
        
        score += reward
        
        clear_output(wait = True)
        env.render()
        
        if done:
            print('Episode finished.')
            print('Goal time:', round(env.goal_time, 2))
            print('Time elapsed:', round(env.time_elapsed, 2))
            print('Score:', round(score, 2))
            time.sleep(2)
            break
            
    if not done:
        print('Goal not reached in {} steps.'.format(max_steps), 'Score:', round(score, 2))
        time.sleep(2)
    
    history.append(score)
    
print('Average score:', round(sum(history) / len(history), 2))

env.close()

# Create and Initialize Q-Table

In [ ]:
action_size = env.action_space.n
state_size = env.observation_space.n

In [ ]:
action_size

In [ ]:
state_size

In [ ]:
q_table = np.zeros((state_size, action_size))

In [ ]:
print(q_table)

# Define Hyperparameters

In [ ]:
num_episodes = 2000
max_steps = 1000

learning_rate = 0.0001
discount_rate = 0.5

exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.001

# Implement Q-Learning Algorithm

In [ ]:
alpha = [.01, .1, .2, .3, .4, .5, .6, .7, .8, .9, .99]
gamma = [.5, .4, .3, .2, .1, .05]
num_episodes = 2000
max_steps = 1000

data = open('RL_Data_Middle_Ojeet_Params.csv', 'w', newline = '')
fieldnames = ['alpha', 'gamma', 'optimal_reward', 'matching_nodes', 'length_diff', 'agent_path']
writer = csv.DictWriter(data, fieldnames = fieldnames)
writer.writeheader()

exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.001

for a in alpha:
    for g in gamma:
        env = PathEnv()
        env.reset()
        
        action_size = env.action_space.n
        state_size = env.observation_space.n
        q_table = np.zeros((state_size, action_size))
        
        rewards_all_episodes = []
        policy_history = []
        path_history = []

        # Q-Learning algorithm
        for episode in range(num_episodes):
            clear_output(wait = True)
            print('alpha:', a)
            print('gamma:', g)
            print('Episode', episode, '/', num_episodes)

            state = env.reset()
            done = False
            rewards_current_episode = 0

            for step in range (max_steps):
                exploration_rate_threshold = random.uniform(0, 1)

                possible_actions = env.get_possible_actions()

                if not possible_actions:
                    action = -1
                else:
                    if exploration_rate_threshold > exploration_rate:
                        action = env.get_best_action()
                    else:
                        action = random.choice(possible_actions)

                n_state, reward, done, info = env.step(action)

                # Update Q-table for Q(s, a)
                q_table[state, action] = q_table[state, action] * (1 - a) + \
                    a * (reward + g * np.max(q_table[n_state, :]))

                state = n_state
                rewards_current_episode += reward

                if done:
                    break

            # Exploration rate decay
            exploration_rate = min_exploration_rate + \
                (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate * episode)

            rewards_all_episodes.append(rewards_current_episode)

            # Save optimal policy
            state = env.reset()
            done = False
            score = 0

            for step in range(max_steps):
                action = env.get_best_action()
                n_state, reward, done, info = env.step(action)
                score += reward
                state = n_state

                if done:
                    if env.goal_reached():
                        path_history.append(env.get_id_path(env.agent_path))

                    policy_history.append(score)
                    break
        
        optimal_reward = policy_history[-1]
        writer.writerow({'alpha' : a, 'gamma' : g, 'optimal_reward' : optimal_reward, 'matching_nodes' : env.get_matching_nodes(sample_paths[2], env.get_id_path(env.agent_path)), 'length_diff' : env.get_length_diff(sample_paths[2], env.get_id_path(env.agent_path)), 'agent_path' : env.get_id_path(env.agent_path)})

data.close()    

# Calculate Average Reward per Hundred Episodes

In [ ]:
rewards_per_hundred_episodes = np.split(np.array(rewards_all_episodes), num_episodes / 100)
count = 100

for r in rewards_per_hundred_episodes:
    print(count, ":", round(sum(r / 100), 1))
    count += 100

# Print Updated Q-Table

In [ ]:
print(q_table)

# Save Updated Q-Table

In [ ]:
filename = 'ENTER_NAME_HERE'

In [ ]:
np.save(filename, q_table)

# Load Q-Table

In [ ]:
q_table = np.load(filename)

# Visualize Optimal Policy Progression

In [ ]:
unique_paths = []

for p in path_history:
    if p not in unique_paths:
        unique_paths.append(p)

In [ ]:
frequency = {}

for p in path_history:
    if unique_paths.index(p) in frequency:
        frequency[unique_paths.index(p)] += 1
    else:
        frequency[unique_paths.index(p)] = 1

In [ ]:
frequency

In [ ]:
vals = list(frequency.values())
top_index = vals.index(max(vals))

print('Most frequent path:', unique_paths[top_index])
print('Frequency:', frequency[top_index])

ox.plot.plot_graph_route(graph, unique_paths[top_index])

In [ ]:
final_path = path_history[len(path_history) - 1]

print('Final path:', final_path)
print('Frequency:', frequency[unique_paths.index(final_path)])

ox.plot.plot_graph_route(graph, final_path)

In [ ]:
for key in frequency:
    print('Path:', unique_paths[key])
    print('Frequency:', frequency[key])
    
    ox.plot.plot_graph_route(graph, unique_paths[key])

# Graph Rewards by Episodes

In [ ]:
plt.plot(policy_history)
plt.ylabel('Reward')
plt.xlabel('Episode')

In [ ]:
policy_history.index(max(policy_history))

In [ ]:
plt.plot(rewards_all_episodes)
plt.ylabel('Reward')
plt.xlabel('Episode')

In [ ]:
rewards_all_episodes.index(max(rewards_all_episodes))

# Replay Episode From History

In [ ]:
step_history = episode_history[0]
max_steps = 1000

state = env.reset()
done = False
score = 0

clear_output(wait = True)
env.render()
time.sleep(0.5)

for step in range(max_steps):
    action = step_history[step]
    n_state, reward, done, info = env.step(action)
    score += reward
    state = n_state

    clear_output(wait = True)
    env.render()

    if done:
        print('Episode finished. Score:', round(score, 3))
        time.sleep(2)
        break

if not done:
    print('Goal not reached in {} steps.'.format(max_steps), 'Score:', round(score, 3))
    time.sleep(2)

env.close()

# Render Environment and Test

In [ ]:
episodes = 1
max_steps = 1000
history = []

for episode in range(1, episodes + 1):
    state = env.reset()
    done = False
    score = 0
    
    clear_output(wait = True)
    env.render()
    time.sleep(0.5)
    
    for step in range(max_steps):
        action = env.get_best_action()
        n_state, reward, done, info = env.step(action)
        score += reward
        state = n_state
        
        clear_output(wait = True)
        env.render()
        print(env.get_q_vals())
        
        if done:
            print('Episode finished.')
            print('Goal time:', round(env.goal_time, 2))
            print('Time elapsed:', round(env.time_elapsed, 2))
            print('Score:', round(score, 2))
            time.sleep(2)
            break
            
    if not done:
        print('Goal not reached in {} steps.'.format(max_steps), 'Score:', round(score, 1))
        time.sleep(2)
    
    history.append(score)

print('Average score:', round(sum(history) / len(history), 2))

env.close()

In [ ]:
print('Sample path:')
ox.plot.plot_graph_route(graph, env.PATH)

print('Agent path:')
ox.plot.plot_graph_route(graph, env.get_id_path(env.agent_path))

In [ ]:
env.get_path_accuracy(env.PATH, env.get_id_path(env.agent_path))